In [ ]:
import requests
from bs4 import BeautifulSoup

 # Descargamos la Web 
url = "http://books.toscrape.com"
respuesta = requests.get(url) 

# BeautifulSoup procesa el HTML 
sopa = BeautifulSoup(respuesta.text, 'Html.parser') # convierte texto HTML ilegible, en un objeto inteligente que se pueda navegar 

# Exposicion: Se busca por ejemplo los titulos de los libros 
titulos = sopa.find_all('h3')
for titulo in titulos:
    print(titulo.get_text())
    

precios = sopa.find_all('p', class_ ='price_color') # sirve para encontrar todos los precios en la clase price_color 
for precio in precios:
    print(precio.get_text()) # el get funciona para conseguir todo el texto que yo le estoy pidiendo  
    